假設有一筆資料 data = { 'size': ['S','M',np.nan,'XL','XL'], 'color': ['red', 'blue', 'blue', 'black', np.nan], 'price': [2100, np.nan, 4500, 7300, 3200], 'quantity': [np.nan, 350, np.nan, 200, 10] } df = pd.DataFrame(data)

(1) 請用ColumnTransformer水平合併器，將數值和類別管道器結合起來。其中數值管道器要做遺漏值處理，用中位數median，再做MinMaxScaler轉換。類別管道器的遺漏值用眾數處理後，再做獨熱編碼。

(2) 請將獨熱編碼的欄位和數值型資料欄位取出，並整合到DataFrame裡

請務必依序回答
print('原始資料')
print('轉換後資料')
print('結合後資料')

In [5]:
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder

# 原始資料
data = {
    'size': ['S', 'M', np.nan, 'XL', 'XL'],
    'color': ['red', 'blue', 'blue', 'black', np.nan],
    'price': [2100, np.nan, 4500, 7300, 3200],
    'quantity': [np.nan, 350, np.nan, 200, 10]
}
df = pd.DataFrame(data)
print('原始資料')
df

原始資料


,size,color,price,quantity
0,S,red,2100.0,NaN
1,M,blue,NaN,350.0
2,NaN,blue,4500.0,NaN
3,XL,black,7300.0,200.0
4,XL,NaN,3200.0,10.0


In [6]:
# 使用ColumnTransformer結合管道器
numeric_features = ['price', 'quantity']
categorical_features = ['size', 'color']

numeric_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', MinMaxScaler())
])

categorical_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder())
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)

# 轉換後資料
transformed_data = preprocessor.fit_transform(df)
print('\n轉換後資料')
print(transformed_data)


轉換後資料
[[0.         0.55882353 0.         1.         0.         0.
  0.         1.        ]
 [0.33653846 1.         1.         0.         0.         0.
  1.         0.        ]
 [0.46153846 0.55882353 0.         0.         1.         0.
  1.         0.        ]
 [1.         0.55882353 0.         0.         1.         1.
  0.         0.        ]
 [0.21153846 0.         0.         0.         1.         0.
  1.         0.        ]]


In [7]:
# 獲取獨熱編碼的欄位和數值型資料欄位，並整合到DataFrame裡
numeric_columns = numeric_features
categorical_columns = preprocessor.transformers_[1][1]['encoder'].categories_
categorical_feature_names = np.concatenate(categorical_columns)

all_columns = np.concatenate((numeric_columns, categorical_feature_names))
transformed_df = pd.DataFrame(transformed_data, columns=all_columns)
print('\n結合後資料')
transformed_df


結合後資料


,price,quantity,M,S,XL,black,blue,red
0,0.000000,0.558824,0.0,1.0,0.0,0.0,0.0,1.0
1,0.336538,1.000000,1.0,0.0,0.0,0.0,1.0,0.0
2,0.461538,0.558824,0.0,0.0,1.0,0.0,1.0,0.0
3,1.000000,0.558824,0.0,0.0,1.0,1.0,0.0,0.0
4,0.211538,0.000000,0.0,0.0,1.0,0.0,1.0,0.0
